## Benchmark for measuring performance of engineered features and models

Prepare data for later analysis

In [1]:
import sys
sys.path.insert(0, '../../bosch_helper')
from bosch_helper import *

%matplotlib inline

### Reload data

In [2]:
x = pd.read_hdf('numeric_all.hdf', 'x')

### Hash value of all numeric rows and `LabelEncode` hash values

In [25]:
hash_numeric = [x.iloc[(n-1)*10**5:n*10**5,:-5].apply(lambda u: hash(tuple(u)), axis=1) for n in range(25)]
hash_numeric = pd.concat(hash_numeric)
hash_numeric = pd.Series(hash_numeric)
hash_numeric.name = 'hash_numeric'

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_results = le.fit_transform(hash_numeric)
hash_numeric = pd.DataFrame(hash_numeric)
hash_numeric['hash_encoded'] = le_results

In [27]:
hash_numeric.drop(['hash_numeric'], axis=1, inplace=True)

In [30]:
hash_value_counts = hash_numeric['hash_encoded'].value_counts()

In [40]:
hash_numeric['count_hash'] = hash_numeric['hash_encoded'].map(hash_value_counts)

In [42]:
hash_numeric.to_csv('hash_numeric.csv.gz', compression='gzip')

In [46]:
hash_numeric.to_hdf('hash_numeric.hdf', 'hash_numeric', complib='blosc:lz4', complevel=9)

### Label encode hash numeric

### Unique value and vlaue counts of each column

In [103]:
column_numeric = x.columns[:-6]

#### Nubmer of unique values for each column

In [113]:
number_unique_values = {c: x.loc[:, c].unique().shape[0] for c in column_numeric}

number_unique_values = pd.Series(number_unique_values, index=column_numeric)

number_unique_values.sort_values(inplace=True)

#### Value counts of each column

In [108]:
unique_counts = {c: x.loc[:, c].value_counts() for c in column_numeric}

Plot one example

In [ ]:
plt.plot(unique_counts['L0_S0_F0'].sort_index().index, unique_counts['L0_S0_F0'].sort_index().values)

### Z-scaled features for each column

In [127]:
import tqdm

##### Directly calculating Z-score has memory error, I will not continue with it to speed up my progress.

In [ ]:
for c in tqdm.tqdm_notebook(column_numeric):
    tmp_mean = x.loc[:, c].mean()
    tmp_std = x.loc[:, c].std()
    x[c+'_zscore'] = (x.loc[:, c] - tmp_mean) / tmp_std